***
# Controlboard for different Data Analytics-Stacks

1. [Vanilla-Jupyter-Datascience-Notebook](#Vanilla-Jupyter-Datascience-Notebook)
2. [Elastic Stack (formerly ELK-Stack)](#Elastic-Stack-(formerly-ELK-Stack))
3. [Neo4j](#Neo4j)
4. [Manipulate your Docker environment](#Manipulate-your-Docker-environment)

***
## Vanilla Jupyter Datascience-Notebook

#### Summary

Your "standard" [Jupyter Notebook for Datascience](https://github.com/jupyter/docker-stacks/tree/master/datascience-notebook), all packages updated. Missing an important library? Then let the AWK team know. 

#### Set some variables first

Specify the Windows directory you want to mount. All **data in the JupyterLab's "work" directory will thus survive the destruction of the container**:

In [ ]:
work_dir = r"C:\Users\kat\Documents\GitHub\example-repo";

# Need to convert Windows path for Docker (linux-based)
work_dir = work_dir.replace("\\", "/").replace(":", "")

Docker image version/"tag" of the Jupyter Datascience-Notebook you want to launch (see the [AWK Repo on Dockerhub](https://hub.docker.com/repository/docker/awkgroupag/datascience-notebook/tags) for the latest tag)

In [ ]:
notebook_version = "e56df3a"

# You might want to change the Docker image used or the port Jupyter runs under (e.g. if you want to run several Jupyters)
notebook = "awkgroupag/datascience-notebook"
port = 8888

#### Start the container (be sure to run the cells above first)

In [ ]:
! sudo docker container run -d -p $port:8888 \
    -e JUPYTER_ENABLE_LAB=yes \
    -v //$work_dir:/home/jovyan/work \
    --name jupyter \
    --network=datalab-network \
    $notebook:$notebook_version 
! echo && echo Waiting for 5 seconds for the container to spin up && echo

from IPython.display import Markdown as md
import time
time.sleep(5)
log = ! sudo docker logs jupyter
url = 'http://127.0.0.1:8888'
for line in log:
    if url in line:
        break
else:
    print(log)
    raise RuntimeError('Did not find URL in the log above')
url = url[:-4] + str(port) + line.split(url, 1)[1]
md(f"**Your Jupyterlab URL is** {url}")

#### Cleaning up
Stop the Vanilla Jupyter Notebook. Container won't be deleted

In [ ]:
! sudo docker stop jupyter

Remove the container

In [ ]:
! sudo docker rm jupyter

***
## Elastic Stack (formerly ELK-Stack)

#### Summary

Elasticsearch, Kibana, Beats, and Logstash. Take data from any source, in any format, then search, analyze, and visualize it in real time.

* **Elasticsearch** is a distributed, RESTful search and analytics engine. As the heart of the Elastic Stack, it centrally stores your data for lightning fast search, fine‑tuned relevancy, and powerful analytics that scale with ease.
* **Kibana** lets you visualize your Elasticsearch data and navigate the Elastic Stack so you can do anything from tracking query load to understanding the way requests flow through your apps.
* **Logstash** is a server-side data processing pipeline that ingests data from a multitude of sources simultaneously, transforms it, and then sends it to your favorite "stash."
* **Beats** is the platform for single-purpose data shippers. They send data from hundreds or thousands of machines and systems to Logstash.

Note that Beats (e.g. Metricbeat or Systembeat) are not included in this stack

#### Connections once the stack has been started
* Direct Kibana browser access: [http://localhost:5601](http://localhost:5601)
* Elasticsearch access for Windows: [http://localhost:9200](http://localhost:9200)
* Access Elasticsearch from **within** a Jupyter container: [http://elasticsearch:9200](http://elasticsearch:9200)
* Logstash access from **within** a Jupyter container: [http://logstash:9600](http://elasticsearch:9600)

#### Create a volume to persist all data

In [ ]:
! sudo docker volume create --name=elasticsearch_data

#### Start the stack
Once pull has completed and containers are running, startup might take 1-2 minutes!

In [ ]:
! sudo docker-compose -f "./elk/docker-compose.yml" -p "elk" up -d

#### Stop and remove the stack (Elasticsearch and Kibana data will be retained)

In [ ]:
! sudo docker-compose -f "./elk/docker-compose.yml" -p "elk" down

#### Delete all Elasticsearch and Kibana data

In [ ]:
! sudo docker volume rm elasticsearch_data

***
## Neo4j

## (under construction)

***
# Manipulate your Docker environment

Show all running and stopped Docker containers

In [ ]:
! sudo docker ps -a

Show all Docker images including their filesizes

In [ ]:
! sudo docker images

Show all volumes (=data volumes if you choose to not mount a Windows directory, for example):

In [ ]:
! sudo docker volume ls

In desperate need to figure out what's eating up your disk space? This command shows where Docker is using disk space:

In [ ]:
! sudo docker system df -v

#### Manipulate a container
Set a container name (or CONTAINER ID) first

In [ ]:
container = "jupyter"

Stop the container

In [ ]:
! sudo docker stop $container

Get the running container's logs saved to the Python variable `logoutput`

In [ ]:
logoutput = ! sudo docker logs $container

Restart an existing (currently stopped) container

In [ ]:
! sudo docker start $container

Remove the container completely

In [ ]:
! sudo docker rm $container

#### Cleaning up and freeing disk space
Remove an image (give either it's name or IMAGE ID)

In [ ]:
image = "test"
! sudo docker image rm $image

Remove all stopped containers at once

In [ ]:
! sudo docker container prune --force

Remove a volume (=data volume, thus potentially deleting your data!):

In [ ]:
volume = "test"
! sudo docker volume rm $volume

**Danger zone**: remove all stopped containers, and all images and all volumes that are currently not associated/mounted with a **running container**. Type the following manually:
* Delete all stopped containers, all "dangling" images, the build cache, any unattached network: ```! sudo docker system prune --force```
* To also delete all currently unused images: ```! sudo docker system prune --all --force```
* To also delete all currently unused volumes (potentially deleting your data!): ```! sudo docker system prune --volumes --force```